# Evaluator Module

In [2]:
#| default_exp evaluator

In [3]:
#| export
import CodeCheckList
import pandas as pd

import CodeCheckList.utils as utils
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker
from CodeCheckList.predictor import Predictor

import statistics
import textdistance

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
#| export
class Evaluator:
    """Evaluator Module to perform all AST Evaluations"""
    def __init__(self, checkpoint: str, language, gpu_available=False):
        self.tokenizer = CodeTokenizer.from_pretrained(checkpoint, language)
        self.masker = Masker(self.tokenizer)
        self.predictor = Predictor.from_pretrained(checkpoint, self.tokenizer, gpu_available)

    def __call__(self, test_set, number_of_predictions: int, masking_rate: float):
        results_dict = self.evaluate_test_set(test_set, number_of_predictions, masking_rate)
        results_dataframe = pd.DataFrame([], columns=[
            'ast_element', 'occurences', 'jaccard', 'sorensen_dice', 'levenshtein', 'jaccard_avg', 'sorensen_dice_avg', 'levenshtein_avg'])
        for result_index, result in enumerate(results_dict):
            results_dataframe.loc[len(results_dataframe.index)] = [self.tokenizer.node_types[result_index], result[0], result[1], result[2], result[3], result[4], result[5], result[6]]
        return results_dataframe
    
    def evaluate_test_set(self, test_set, number_of_predictions: int, masking_rate: float):
        results_dict = []
        for node_type in self.tokenizer.node_types:
            results_dict.append([0,                                           #ocurrences
                                [[] for i in range(0,number_of_predictions)], #jaccard per prediction
                                [[] for i in range(0,number_of_predictions)], #sorensen_dice per prediction
                                [[] for i in range(0,number_of_predictions)], #levenshtein per prediction
                                [0 for i in range(0,number_of_predictions)],  #avg jaccard per prediction
                                [0 for i in range(0,number_of_predictions)],  #avg sorensen_dice per prediction
                                [0 for i in range(0,number_of_predictions)],  #avg levenshtein per prediction
                                ])
        for sample_index, sample in enumerate(test_set):
            print('--------evaluating sample:'+str(sample_index)+' --------')
            for node_type_idx, node_type in enumerate(self.tokenizer.node_types):
                node_type_results = self.evaluate_node_type_on_snippet(sample['whole_func_string'], node_type_idx, number_of_predictions, masking_rate)
                if(len(node_type_results)>0):
                    results_dict[node_type_idx][0] += node_type_results[0][0]
                    for prediction_number_index in range(0, number_of_predictions):
                        results_dict[node_type_idx][1][prediction_number_index].append(node_type_results[prediction_number_index][1])
                        results_dict[node_type_idx][2][prediction_number_index].append(node_type_results[prediction_number_index][2])
                        results_dict[node_type_idx][3][prediction_number_index].append(node_type_results[prediction_number_index][3])
                        results_dict[node_type_idx][4][prediction_number_index] = round(statistics.mean(results_dict[node_type_idx][1][prediction_number_index]),3)
                        results_dict[node_type_idx][5][prediction_number_index] = round(statistics.mean(results_dict[node_type_idx][2][prediction_number_index]),3)
                        results_dict[node_type_idx][6][prediction_number_index] = round(statistics.mean(results_dict[node_type_idx][3][prediction_number_index]),3)
        return results_dict
        
    def evaluate_node_type_on_snippet(self, source_code: str, target_node_type_idx: int, number_of_predictions: int, masking_rate: float):
        results=[]

        source_code_tree = self.tokenizer.parser.parse(bytes(source_code, "utf8")).root_node
        source_code_nodes = []
        utils.find_nodes(source_code_tree,self.tokenizer.node_types[target_node_type_idx], source_code_nodes)
        if len(source_code_nodes) == 0:
            return results

        masked_code_encoding = self.masker(source_code, self.tokenizer(source_code), target_node_type_idx, masking_rate)
        predictions = self.predictor(masked_code_encoding, self.tokenizer.tokenizer(source_code, return_tensors='pt'), number_of_predictions)  

        for prediction_number in range(0, number_of_predictions):
            predicted_code = predictions[prediction_number]
            jaccard_similarity = 0        #the closest to 1, the best
            sorensen_dice_similarity = 0  #the closest to 1, the best
            levenshtein_similarity = 0    #the closest to 1, the best

            if utils.is_balanced_snippet(predicted_code):
                predicted_code_tree = self.tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node
                predicted_code_types = utils.get_node_type_list(predicted_code_tree)
                source_code_types = utils.get_node_type_list(source_code_tree)
                jaccard_similarity = textdistance.jaccard.normalized_similarity(predicted_code_types,source_code_types)
                sorensen_dice_similarity = textdistance.sorensen_dice.normalized_similarity(predicted_code_types, source_code_types)
                levenshtein_similarity = textdistance.levenshtein.normalized_similarity(predicted_code_types,source_code_types)
            results.append([len(source_code_nodes), jaccard_similarity, sorensen_dice_similarity, levenshtein_similarity])
        return results


## Full Pipeline

### Download Grammar

In [5]:
from CodeCheckList import loader

"""define language"""
python_language = "python"

languages = [python_language]

loader.download_grammars(languages)

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/CodeCheckList/grammars


### Load Model

In [6]:

"""define the model checkpoint"""
checkpoint = "huggingface/CodeBERTa-small-v1"

### Create Modules

In [7]:
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker

#create code tokenizer 
bert_tokenizer = CodeTokenizer.from_pretrained(checkpoint, python_language)

#create code masker
code_masker = Masker(bert_tokenizer)

### Node Types

In [8]:
print(bert_tokenizer.node_types)

['primary_expression', '~', 'ERROR', 'decorated_definition', 'try', '<>', 'raise', 'or', '^=', '[', 'default_parameter', 'keyword_separator', 'generator_expression', '^', 'pass', 'function_definition', 'global_statement', 'ellipsis', 'parenthesized_list_splat', 'block', 'from', '.', 'dictionary_splat_pattern', 'global', 'string', ':', 'elif_clause', '|=', ')', 'case_clause', 'exec_statement', 'pattern', 'escape_sequence', 'try_statement', 'type', 'call', '//=', 'type_conversion', 'none', 'concatenated_string', '__future__', 'pair', '//', 'decorator', '"', 'nonlocal_statement', 'class_definition', 'except_clause', '=', '}', 'dotted_name', 'import', '->', 'set_comprehension', 'attribute', '<<', '*', '%=', 'unary_operator', '<', 'list_comprehension', '>>', 'future_import_statement', 'positional_separator', 'del', 'augmented_assignment', 'not', 'named_expression', 'parameters', 'subscript', 'dictionary_splat', 'lambda', 'raise_statement', 'assignment', 'dictionary', 'continue_statement', '

### Encodings

In [9]:
"""example source code"""

code = "def multiply_numbers(a,b):\n    return a*b"
#code = "def scale(self, center=True, scale=True):\n        \"\"\"\nthe the\n\n\n                                                                                                                                                          _\n                     ____________=_=_===========________===______________________________==_____________________\n_______\n____\n\n___\n\n\n\n\n\n\n\n\n        return return)"
#code = "def hello_world(a,b):\n    print('hello world')"
#code = "def __ordered_values_by_indexes(self, data, inds): \"\"\" Return values (intensities) by indexes. Used for multiscale graph cut. data = [[0 1 1], [0 2 2], [0 2 2]] inds = [[0 1 2], [3 4 4], [5 4 4]] return: [0, 1, 1, 0, 2, 0] If the data are not consistent, it will take the maximal value \"\"\" # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
#code = "def __ordered_values_by_indexes(self, data, inds):  # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
target_node_type = "identifier"

#encoding 
source_code_encoding = bert_tokenizer(code)

#masking
masked_code_encoding = code_masker(code, bert_tokenizer(code), bert_tokenizer.node_types.index(target_node_type), 1)

assert len(source_code_encoding['input_ids']) == len(masked_code_encoding['input_ids'])

#masked code
masked_code = bert_tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == bert_tokenizer.tokenizer.bos_token_id or 
            token_id == bert_tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))

print(masked_code)

def<mask><mask><mask>(<mask>,<mask>):
    return<mask>*<mask>


### Code Prediction

In [10]:
from CodeCheckList.predictor import Predictor

predictor = Predictor.from_pretrained(checkpoint, bert_tokenizer)
predictions = predictor(masked_code_encoding, bert_tokenizer.tokenizer(code, return_tensors='pt'), 5)

### Evaluation

In [11]:
import CodeCheckList.utils as utils

prediction_number = 0
print('------------- CODE -------------')
print(code)
print('\n---------- MASKED -------------')
print(masked_code)
print('\n--------- PREDICTED -----------')
predicted_code = predictions[prediction_number]
print(predicted_code)
print('\n--------- AST COMPARE -----------')
filtered_nodes = []
filtered_nodes_predict = []
utils.find_nodes(bert_tokenizer.parser.parse(bytes(code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes)
utils.find_nodes(bert_tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes_predict)
print(len(filtered_nodes))
print(len(filtered_nodes_predict))
#base the evaluation on size comparison

------------- CODE -------------
def multiply_numbers(a,b):
    return a*b

---------- MASKED -------------
def<mask><mask><mask>(<mask>,<mask>):
    return<mask>*<mask>

--------- PREDICTED -----------
def __function(name, value):
    return f*args

--------- AST COMPARE -----------
5
5


## Testing

In [12]:
from datasets import load_dataset 
import CodeCheckList.utils as utils


evaluator = Evaluator(checkpoint, python_language)

max_token_number = bert_tokenizer.tokenizer.max_len_single_sentence
print(max_token_number)

test_set = load_dataset("code_search_net", split='test')
test_set = test_set.filter(lambda sample: True if sample['language']== python_language
            and len(sample['func_code_tokens']) <= max_token_number
            and len(bert_tokenizer.tokenizer(sample['whole_func_string'])['input_ids']) <= max_token_number else False, num_proc=1)

test_set = utils.get_random_sub_set_test_set(utils.get_test_sets(load_dataset("code_search_net", split='test'), "python", evaluator.tokenizer.tokenizer.max_len_single_sentence, evaluator.tokenizer), 5)

len(test_set)

510


No config specified, defaulting to: code_search_net/all
Found cached dataset code_search_net (/home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)
Parameter 'function'=<function <lambda> at 0x7f50b991b9a0> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 101/101 [00:24<00:00,  4.09ba/s]
No config specified, defaulting to: code_search_net/all
Found cached dataset code_search_net (/home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b

5

In [15]:
number_of_predictions = 3
checkpoint = "huggingface/CodeBERTa-small-v1"
python_language = "python"
masking_rate = 1

evaluator = Evaluator(checkpoint, python_language, gpu_available=False)

results_dataframe = evaluator(test_set, number_of_predictions, masking_rate)

results_dataframe.sort_values(by=['occurences'], ascending=False)


--------evaluating sample:0 --------
--------evaluating sample:1 --------
--------evaluating sample:2 --------
--------evaluating sample:3 --------
--------evaluating sample:4 --------


,ast_element,occurences,jaccard,sorensen_dice,levenshtein,jaccard_avg,sorensen_dice_avg,levenshtein_avg
113,identifier,135,"[[0.9851851851851852, 0.9365079365079365, 0.42...","[[0.9925373134328358, 0.9672131147540983, 0.59...","[[0.9851851851851852, 0.9516129032258065, 0.45...","[0.469, 0.519, 0.456]","[0.511, 0.556, 0.509]","[0.478, 0.519, 0.456]"
44,"""",42,"[[0.8012048192771084, 1.0, 1.0, 0, 0], [0.8012...","[[0.8896321070234113, 1.0, 1.0, 0, 0], [0.8896...","[[0.8012048192771084, 1.0, 1.0, 0, 0], [0.8012...","[0.56, 0.56, 0.556]","[0.578, 0.578, 0.577]","[0.56, 0.56, 0.557]"
149,(,35,"[[0.9851851851851852, 1.0, 1.0, 0, 0], [0.95, ...","[[0.9925373134328358, 1.0, 1.0, 0, 0], [0.9743...","[[0.9851851851851852, 1.0, 1.0, 0, 0], [0.9357...","[0.597, 0.584, 0.562]","[0.599, 0.592, 0.579]","[0.597, 0.581, 0.571]"
28,),35,"[[1.0, 1.0, 1.0, 0, 0], [0.9097222222222222, 1...","[[1.0, 1.0, 1.0, 0, 0], [0.9527272727272728, 1...","[[1.0, 1.0, 1.0, 0, 0], [0.8802816901408451, 1...","[0.6, 0.573, 0.584]","[0.6, 0.586, 0.592]","[0.6, 0.568, 0.583]"
21,.,26,"[[1.0, 1.0, 1.0, 0, 0], [0.9851851851851852, 1...","[[1.0, 1.0, 1.0, 0, 0], [0.9925373134328358, 1...","[[1.0, 1.0, 1.0, 0, 0], [0.9481481481481482, 1...","[0.6, 0.597, 0.565]","[0.6, 0.599, 0.581]","[0.6, 0.59, 0.567]"
...,...,...,...,...,...,...,...,...
77,exec,0,"[[], [], []]","[[], [], []]","[[], [], []]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"
78,pass_statement,0,"[[], [], []]","[[], [], []]","[[], [], []]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"
79,finally_clause,0,"[[], [], []]","[[], [], []]","[[], [], []]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"
80,print_statement,0,"[[], [], []]","[[], [], []]","[[], [], []]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"
